In [ ]:
from pyabc import (ABCSMC,
                   RV, Distribution,
                   IntegratedModel, ModelResult,
                   MedianEpsilon,
                   LocalTransition)
from pyabc.sampler import SingleCoreSampler
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tempfile
import pandas as pd
import scipy as sp
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))

In [ ]:
n_steps = 30

In [ ]:
def simulate(step_size):
    trajectory = sp.zeros(n_steps)
    for t in range(1, n_steps):
        xi = sp.rand()
        trajectory[t] = trajectory[t-1] + xi * step_size
    return trajectory

In [ ]:
def distance(trajectory_1, trajectory_2):
    return sp.absolute(trajectory_1 - trajectory_2).sum()

In [ ]:
gt_step_size = 5

In [ ]:
gt_trajectory = simulate(gt_step_size)
trajectoy_2 = simulate(2)

dist_1_2 = distance(gt_trajectory, trajectoy_2)

plt.plot(gt_trajectory,
    label="Step size = {} (Ground Truth)".format(gt_step_size))
plt.plot(trajectoy_2,
    label="Step size = 2")
plt.legend();
plt.title("Distance={:.2f}".format(dist_1_2));

In [ ]:
class MyStochasticProcess(IntegratedModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_early_stopped = 0
    
    def integrated_simulate(self, pars, eps):
        cumsum = 0
        trajectory = sp.zeros(n_steps)
        for t in range(1, n_steps):
            xi = sp.rand()
            next_val = trajectory[t-1] + xi * pars["step_size"]
            cumsum += abs(next_val - gt_trajectory[t])
            trajectory[t] = next_val
            if cumsum > eps:
                self.n_early_stopped += 1
                return ModelResult(accepted=False)
            
        return ModelResult(accepted=True,
                           distance=cumsum,
                           sum_stats={"trajectory": trajectory})

In [ ]:
prior = Distribution(step_size=RV("uniform", 0 , 10))

In [ ]:
model = MyStochasticProcess()

In [ ]:
abc = ABCSMC(models=model,
             parameter_priors=prior,
             distance_function=None,
             sampler=SingleCoreSampler(),
             population_specification=30,
             transitions=LocalTransition(k_fraction=.2),
             eps=MedianEpsilon(300, median_multiplier=0.7))

In [ ]:
abc.new(db_path)

In [ ]:
h = abc.run(minimum_epsilon=40, max_nr_populations=3)

In [ ]:
model.n_early_stopped

In [ ]:
df = pd.DataFrame({"step_size": sp.linspace(0, 10, 300)})
kde = LocalTransition(k_fraction=.2)
for t in range(h.max_t+1):
    kde.fit(*h.get_distribution(m=0, t=t))
    df["t={}".format(t)] = kde.pdf(df[["step_size"]])

In [ ]:
ax = df.plot(x="step_size");
ax.axvline(gt_step_size, color="k", linestyle="dashed");